In [1]:
# Typically would use pandas for something like this, but since the dataset is small, sticking to vanilla python

In [14]:
import csv
from dataclasses import dataclass
from datetime import datetime
import os 
import pprint
pp = pprint.PrettyPrinter(indent=4)

datafolder = 'data-dev/'
# datafolder = 'data-prod/'

In [15]:
filename = datafolder + 'ingredients v1.csv'

In [16]:
raw_food_data = []
with open(filename, newline='', encoding='utf-8-sig') as csvfile:
    file_reader = csv.DictReader(csvfile, delimiter=',')
    headers = next(file_reader)
    for row in file_reader:
        raw_food_data.append(row)
    
pp.pprint(raw_food_data[0].keys())
pp.pprint(raw_food_data[0])

dict_keys(['id', 'asins', 'brand', 'categories', 'dateAdded', 'dateUpdated', 'ean', 'features.key', 'features.value', 'manufacturer', 'manufacturerNumber', 'name', 'sizes', 'upc', 'weight', ''])
{   '': '',
    'asins': 'B008VT0W8C,B0092F8OJ8',
    'brand': 'McCormick',
    'categories': 'Grocery & Gourmet Food,Food,Grocery',
    'dateAdded': '2016-02-05T21:08:01Z',
    'dateUpdated': '2017-04-12T13:54:04Z',
    'ean': '52100018164',
    'features.key': 'Ingredients',
    'features.value': 'Salt,Sugar,Molasses (Refinery Syrup, Molasses, Caramel '
                      'Color),Spices (Including Black Pepper),Garlic '
                      'Onion,Tapioca Maltodextrin,Bacon Fat and Cooked Bacon '
                      '(Cured with Water, Salt, Sodium Erythorbate, Sodium '
                      'Nitrate),Silicon Dioxide (To Make Free '
                      'Flowing),Autolyzed Yeast,Sunflower Oil,Corn '
                      'Maltodextrin,Vinegar,Extractives of Paprika,and Natural '
      

In [17]:
# For brevity, just discarding any that don't match this simple validator
pruned_food_data = []
print('raw_food_data Number of Entries:','\t', len(raw_food_data) )

def has_present_data(item: dict) -> bool:
    if item['name'] == '': return False
    if item['brand'] == '': return False
    if item['features.key'] != 'Ingredients': return False
    if item['features.value'] == '': return False
    if item['features.value'] == 'FALSE': return False
    if item['manufacturer'] == '': return False
    if item['upc'] == '': return False
    return True

pruned_food_data = [ item for item in raw_food_data if has_present_data(item)]

print('pruned_food_data Number of Entries:', '\t', len(pruned_food_data))

raw_food_data Number of Entries: 	 9999
pruned_food_data Number of Entries: 	 6265


In [18]:
import re

def data_in_parens(ingredients):
    return re.findall(r'[(](.*?)[)]', ingredients)
    
def split_on_comma_if_not_in_parens(ingredients):
    return [i.strip() for i in re.split(r',(?![^\(]*[\)])', ingredients)]

def print_sample(item):
    ingredients = item['features.value']
    print('Raw Text:')
    print('')
    print(ingredients)
    print('')
    print('')
    print('Split on Comma, Excluding Parens')
    print(split_on_comma_if_not_in_parens(ingredients))
    print('')
    print('')
    print('Data in Parens')
    print(data_in_parens(ingredients))

print_sample(pruned_food_data[0])

Raw Text:

Salt,Sugar,Molasses (Refinery Syrup, Molasses, Caramel Color),Spices (Including Black Pepper),Garlic Onion,Tapioca Maltodextrin,Bacon Fat and Cooked Bacon (Cured with Water, Salt, Sodium Erythorbate, Sodium Nitrate),Silicon Dioxide (To Make Free Flowing),Autolyzed Yeast,Sunflower Oil,Corn Maltodextrin,Vinegar,Extractives of Paprika,and Natural Flavor (Including Smoke)


Split on Comma, Excluding Parens
['Salt', 'Sugar', 'Molasses (Refinery Syrup, Molasses, Caramel Color)', 'Spices (Including Black Pepper)', 'Garlic Onion', 'Tapioca Maltodextrin', 'Bacon Fat and Cooked Bacon (Cured with Water, Salt, Sodium Erythorbate, Sodium Nitrate)', 'Silicon Dioxide (To Make Free Flowing)', 'Autolyzed Yeast', 'Sunflower Oil', 'Corn Maltodextrin', 'Vinegar', 'Extractives of Paprika', 'and Natural Flavor (Including Smoke)']


Data in Parens
['Refinery Syrup, Molasses, Caramel Color', 'Including Black Pepper', 'Cured with Water, Salt, Sodium Erythorbate, Sodium Nitrate', 'To Make Free Flowin

In [19]:
print_sample(pruned_food_data[1])

Raw Text:

Salt, Yellow 5 Lake, Tricalcium Phosphate And Artificial Butter Flavor


Split on Comma, Excluding Parens
['Salt', 'Yellow 5 Lake', 'Tricalcium Phosphate And Artificial Butter Flavor']


Data in Parens
[]


In [20]:
print_sample(pruned_food_data[2])

Raw Text:

Mechanically hulled seasame seeds.Allergy Information: Packed in a facility that processes wheat, flour, peanuts and tree nuts.,Mechanically hulled seasame seeds.Allergy Information: Packed in a facility that processes wheat,flour,peanuts and tree nuts.


Split on Comma, Excluding Parens
['Mechanically hulled seasame seeds.Allergy Information: Packed in a facility that processes wheat', 'flour', 'peanuts and tree nuts.', 'Mechanically hulled seasame seeds.Allergy Information: Packed in a facility that processes wheat', 'flour', 'peanuts and tree nuts.']


Data in Parens
[]


In [21]:
# Splitting on Comma, Excluding Parens seems to be the best 'quick and dirty' approach, 
# can be later cleaned via the web interface

In [22]:
# UPC has some formatting issues, so remove commas
#
# https://en.wikipedia.org/wiki/Universal_Product_Code
#   according to this article, upc code should be 12 digits
# 

columns = ['name', 'brand', 'upc', 'ingredients']
write_to_csv = []
for food in pruned_food_data:
    food_data = {
        "name": food['name'],
        "brand": food['brand'],
        "upc": int(float(food['upc'].replace(',',''))), # parse exponent notation since upc is an int
        "ingredients": ','.join(split_on_comma_if_not_in_parens(food['features.value']))
    }
    write_to_csv.append(food_data)

In [23]:
# Save to a new csv
filename = datafolder + 'ingredients v2.csv'
try:
    with open(filename, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()
        for data in write_to_csv:
            writer.writerow(data)
except IOError:
    print("I/O error")